# Detection and Classification of Rice Leaf Diseases using Convolutional Neural Networks

In [ ]:
pip install utils


In [5]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras import models, layers

import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
%matplotlib inline
from keras.utils import np_utils
from tensorflow import keras

In [9]:
riceleaf = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\Home\Desktop\Rice Leaf Disease")

Found 5932 files belonging to 5 classes.


In [10]:
#riceleaf = tf.keras.models.image_dataset_from_directory(r"C:\Users\Home\Desktop\Rice Leaf Disease")

In [11]:
datagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest", validation_split=0.3)
batch_size = 32

train_generator = datagen.flow_from_directory(
    r"C:\Users\Home\Desktop\Rice Leaf Disease",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    r"C:\Users\Home\Desktop\Rice Leaf Disease",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 4153 images belonging to 5 classes.
Found 1779 images belonging to 5 classes.


## Modeling
For the modeling, we used different pre-trained models. Six architectures were used:

1.VGG16 base for feature extraction and 3-dense layers for classification.

2.VGG16 base with only the first 3 blocks frozen and 3-dense layers.

3.VGG19 base and 3-dense layers.

4.XCeption base and 3-dense layers.

5.ResNet base and 3-dense layers.

6.5 convolutional layers and 2-dense layer.

## VGG16

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint = ModelCheckpoint('VGG16.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [13]:
datagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
# datagen = ImageDataGenerator(rescale=1./255)

def extract_features(trainorval, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count, 4))
    if trainorval=="training":
        generator = train_generator
    else:
        generator = val_generator
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(preprocess_input(inputs_batch))
        try:
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        except ValueError:
            break
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [15]:
conv_base.trainable = True
for layer in conv_base.layers:
    layer.trainable = True

In [16]:
train_features, train_labels = extract_features('training', 2319)
validation_features, validation_labels = extract_features('validation', 991)

train_features = np.reshape(train_features, (2319, 7 * 7 * 512))
validation_features = np.reshape(validation_features, (991, 7 * 7 * 512))

In [17]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=(25088)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))  #Removing 50% of the weights!
model.add(layers.Dense(4, activation='softmax'))

In [18]:
from tensorflow.keras.optimizers import Adam

In [19]:
INIT_LR = 1e-1
EPOCHS = 30

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

C:\Users\Home\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
import os
from tensorflow.keras.optimizers import Adam

In [21]:
os.environ["PATH"] += os.pathsep + 'C:\\Users\\1000246125\\AppData\\Local\\Continuum\\anaconda3\\Library\\bin'
model = Sequential()
model.add(Dense(2, input_dim=7*7*512, activation='relu'))
model.add(Dense(4, activation='softmax'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [25]:
INIT_LR = 1e-1
EPOCHS = 30

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('VGG16.h5', verbose=1, monitor='val_acc', save_best_only=True, mode='auto') 

history = model.fit(train_features, train_labels,
                    epochs=7,
                    batch_size=batch_size,
                    validation_data=(validation_features, validation_labels),
                    callbacks=[checkpoint])

Epoch 1/7
73/73 [==============================] - 2s 16ms/step - loss: 0.0699 - accuracy: 0.9996 - val_loss: 2.0294e-04 - val_accuracy: 1.0000
Epoch 2/7
73/73 [==============================] - 1s 8ms/step - loss: 1.7389e-04 - accuracy: 1.0000 - val_loss: 1.4499e-04 - val_accuracy: 1.0000
Epoch 3/7
73/73 [==============================] - 1s 8ms/step - loss: 1.2951e-04 - accuracy: 1.0000 - val_loss: 1.1097e-04 - val_accuracy: 1.0000
Epoch 4/7
73/73 [==============================] - 1s 8ms/step - loss: 1.0122e-04 - accuracy: 1.0000 - val_loss: 8.8660e-05 - val_accuracy: 1.0000
Epoch 5/7
73/73 [==============================] - 1s 8ms/step - loss: 8.2213e-05 - accuracy: 1.0000 - val_loss: 7.3259e-05 - val_accuracy: 1.0000
Epoch 6/7
73/73 [==============================] - 1s 8ms/step - loss: 6.8829e-05 - accuracy: 1.0000 - val_loss: 6.2154e-05 - val_accuracy: 1.0000
Epoch 7/7
73/73 [==============================] - 1s 8ms/step - loss: 5.8997e-05 - accuracy: 1.0000 - val_loss: 5.3830e-

# VGG16 (Frozen first 3)

In [26]:
datagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest", validation_split=0.3)
batch_size = 32

train_generator = datagen.flow_from_directory(
    r"C:\Users\Home\Desktop\Rice Leaf Disease",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    r"C:\Users\Home\Desktop\Rice Leaf Disease",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 4153 images belonging to 5 classes.
Found 1779 images belonging to 5 classes.


In [27]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [31]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))  #Removing 50% of the weights!
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

INIT_LR = 1e-3
EPOCHS = 30

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('VGG16_frozen3.h5', verbose=1, monitor='val_acc', save_best_only=True, mode='auto') 

history = model.fit_generator(train_generator,
                    epochs=2,
                    steps_per_epoch=7*7*512 / batch_size,
                    validation_data=val_generator,
                    callbacks=[checkpoint])